# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [73]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, accuracy_score

In [37]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [38]:
spaceship.shape

(8693, 14)

**Check for data types**

In [39]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

**Check for missing values**

In [40]:
spaceship.values.any()

np.True_

In [41]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [42]:
df = spaceship.dropna()

In [43]:
df.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [44]:
df.shape

(6606, 14)

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [45]:
df["Cabin"].unique()

array(['B/0/P', 'F/0/S', 'A/0/S', ..., 'G/1499/S', 'G/1500/S', 'E/608/S'],
      dtype=object)

In [46]:
df["Cabin"].value_counts()

Cabin
G/1476/S    7
E/13/S      7
C/137/S     7
G/734/S     7
B/11/S      7
           ..
E/233/S     1
E/209/P     1
G/548/S     1
D/108/P     1
B/153/P     1
Name: count, Length: 5305, dtype: int64

In [47]:
df['Cabin'] = df['Cabin'].str[0]

/var/folders/bp/q0jpdzq126j54kx_lzjc4dcm0000gn/T/ipykernel_90686/3666398392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cabin'] = df['Cabin'].str[0]


In [48]:
print(df["Cabin"].unique())

['B' 'F' 'A' 'G' 'E' 'C' 'D' 'T']


In [49]:
df["Cabin"].value_counts()

Cabin
F    2152
G    1973
E     683
B     628
C     587
D     374
A     207
T       2
Name: count, dtype: int64

- Drop PassengerId and Name

In [50]:
df_clean = df.drop(["PassengerId", "Name"], axis=1)

print(df_clean.head())

  HomePlanet CryoSleep Cabin  Destination   Age    VIP  RoomService  \
0     Europa     False     B  TRAPPIST-1e  39.0  False          0.0   
1      Earth     False     F  TRAPPIST-1e  24.0  False        109.0   
2     Europa     False     A  TRAPPIST-1e  58.0   True         43.0   
3     Europa     False     A  TRAPPIST-1e  33.0  False          0.0   
4      Earth     False     F  TRAPPIST-1e  16.0  False        303.0   

   FoodCourt  ShoppingMall     Spa  VRDeck  Transported  
0        0.0           0.0     0.0     0.0        False  
1        9.0          25.0   549.0    44.0         True  
2     3576.0           0.0  6715.0    49.0        False  
3     1283.0         371.0  3329.0   193.0        False  
4       70.0         151.0   565.0     2.0         True  


In [51]:
df_clean["HomePlanet"].value_counts()

HomePlanet
Earth     3566
Europa    1673
Mars      1367
Name: count, dtype: int64

- For non-numerical columns, do dummies.

In [52]:
non_numeric_cols = df_clean.select_dtypes(include = ["object", "bool"]).columns
print(non_numeric_cols)

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP',
       'Transported'],
      dtype='object')


In [53]:
df2 = pd.get_dummies(df_clean, columns=non_numeric_cols, drop_first=True) #

In [54]:
df2.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Transported_True
0,39.0,0.0,0.0,0.0,0.0,0.0,True,False,False,True,False,False,False,False,False,False,False,True,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,False,False,False,False,False,False,False,True,False,False,False,True,False,True
2,58.0,43.0,3576.0,0.0,6715.0,49.0,True,False,False,False,False,False,False,False,False,False,False,True,True,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,True,False,False,False,False,False,False,False,False,False,False,True,False,False
4,16.0,303.0,70.0,151.0,565.0,2.0,False,False,False,False,False,False,False,True,False,False,False,True,False,True


In [55]:
df2.isnull().sum()

Age                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
HomePlanet_Europa            0
HomePlanet_Mars              0
CryoSleep_True               0
Cabin_B                      0
Cabin_C                      0
Cabin_D                      0
Cabin_E                      0
Cabin_F                      0
Cabin_G                      0
Cabin_T                      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
VIP_True                     0
Transported_True             0
dtype: int64

**Perform Train Test Split**

In [62]:
cols_to_drop = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [63]:
df2_clean = df2.drop(columns=cols_to_drop)

In [64]:
x = df2_clean.drop("Transported_True", axis=1)
y = df2_clean["Transported_True"]

In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [71]:
x_train.head(4)

,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
7832,True,False,False,False,False,False,False,False,False,False,False,False,False
5842,True,False,False,False,True,False,False,False,False,False,False,True,False
3928,False,True,True,False,False,False,False,True,False,False,False,False,False
4091,False,False,True,False,False,False,False,False,True,False,False,False,False


In [72]:
x_train.shape

(5284, 13)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [95]:
knn = KNeighborsClassifier(n_neighbors=10)

In [96]:
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10)

- Evaluate your model's performance. Comment it

In [97]:
pred_train = knn.predict(x_train)
accuracy_train = accuracy_score(y_train, pred_train)
print("TRAIN DATA Accuracy: ", accuracy_train)

print("--------------------------")

pred_test = knn.predict(x_test)
accuracy_test = accuracy_score(y_test, pred_test)
print("TEST DATA Accuracy: ", accuracy_test)


TRAIN DATA Accuracy:  0.7115821347464042
--------------------------
TEST DATA Accuracy:  0.7110438729198184


We trained a K-Nearest Neighbors (KNN) classifier on the dataset to predict Transported_True. We removed the continuous columns (Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck) and kept only the categorical/dummy features.

**Results with KNN (n_neighbors=10):**
- Training Accuracy: 71.16%
- Test Accuracy: 71.10%

**Observations:**
- The model shows consistent performance between training and test sets, indicating it is not overfitting.
- Accuracy is limited (~71%), suggesting that the categorical features alone may not provide enough information for better predictions.
- Adjusting n_neighbors did not improve accuracy:
- Fewer neighbors → accuracy ~70%
- More neighbors → accuracy ~71%

**Next steps to potentially improve performance:**
- Include some continuous features, scaled appropriately.
- Try other classifiers like RandomForestClassifier or LogisticRegression.
- Perform feature engineering to create new informative variables from existing data.